# Distilling Object Detectors with Fine-grained Feature Imitation  
- paper review

## Introduction    
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/Distilling-Object-Detectors/figures/figure1.png?raw=true" width=500>

- classification model은 image의 global feature를 보는 반면 object detection model은 local feature만 보면서 학습함  
- 그래서 direct로 full feature imitation은 거대한 noisy가 생겨 object detection 학습에 방해가 될 수 있음  
- detector는 local object region에 더 신경을 쓰기 때문에 object 근처의 anchor location에서 feature reponse의 불일치는 모델이 object를 탐지하는 방법에 대한 중요한 정보를 전달함  
- 그래서 우리는 object anchor location 근처의 유의마한 정보를 찾기 위해 anchor priors와 ground truth bounding box를 이용한 새로운 mechanism을 개발한다  
- 우리는 이를 fine-grained feature imitation이라고 칭한다  
- challenge  
    - 1. vanilla knowledge distillation처럼 teacher model의 softened output에 의존하지 않지만 teacher network의 high level feature response의 inter-location discrepancy에 의존한다  
    - 2. Fine-grained feature imitation을 사용함으로써 성능을 개선시킨다  
    - 3. 우리의 method는 성능을 저하시키는 background area에 대한 noisy를 피한다  

## Method  
### Imitation region estimation  
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/Distilling-Object-Detectors/figures/figure2.png?raw=true" width=500>  
  
  
- local feature region라는 것을 정의하고 연구하기 위해 우리는 ground truth bounding box와 anchor prior를 사용해 각 independent image에 대한 mask I로 이 region을 계산하고 thresholding factor $\psi$로 region의 size를 제어한다  
- 각 ground truth box에 대해, 우리는 $W\times H\times K $ IOU map $m$ 형태로 anchor와 box의 IOU를 계산한다  
- 이때 $W$와 $H$는 feature map의 width, height를 의미하고 $K$는 preset anchor box를 의미한다  
- 그러면 우리는 largest IOU를 찾을 수 있으며 이를 $M=max(m)$이라 정의하며 이와 thresholding factor $\psi$를 곱해 filter thresohld $F=\psi * M$를 얻는다  
- ground truth box들을 연결하고 mask들을 결합하면 final fine-grained imitation mask I를 얻을 수 있다  
- $\psi=0$일 때, generated mask는 feature map의 모든 location을 포함하며 $\psi=1$일 때 모든 location을 포함하지 않는다  

### Fine-grained feature imitation  
- imitation을 수행하기 위해 figure 2처럼 student와 teacher의 feature response 사이의 distance metric을 계산하기 전에 student model 이후에 full convolution adaptation layer를 추가한다  
- reasons:  
    - 1. student와 teacher의 feature가 호환이 되도록 하기 위해서, distance metric을 계산하기 위해  
    - 2. student와 teacher의 feature가 호환이 될 때 teacher feature를 approximate하도록 강요하는 것이 minor gains를 얻게 됨  
    
- teacher network의 knowledge를 학습하기 위한 objective는 다음과 같이 정의한다  
$$l=\sum_{c=1}^C \left(f_{adap}\left(s\right)_{ijc}-t_{ijc}\right)^2$$  

- imitation mask I를 포함하면  
$$L_{imitation}=\frac{1}{2N_p}\sum_{i=1}^W\sum_{j=1}^H\sum_{c=1}^CI_{ij}\left(f_{adap}\left(s\right)_{ijc}-t_{ijc}\right)^2$$  
- where $N_p=\sum_{i=1}^W\sum_{j=1}^HI_{ij}$ and $N_p$ is number of positive points in mask, $f_{adap}\left(\cdot)$ is the adaptation function  
- 최종적으로 student model의 training loss는 다음과 같다  
$$L=L_{gt}+\lambda L_{imitation}$$  
- where $L_gt$ is the detection training loss and $\lambda$ is imitation loss weight balancing factor  

## Experiments  
- encoder network로 Shufflenet, ResNet, VGG 등을 적용  
- channel의 수를 줄이거나 block의 수를 줄이는 등의 방법을 이용해 lightweight detector를 생성 (student)  
- detector로 Faster R-CNN을 사용해서 실험  

<img src = "https://github.com/Sangh0/Object-Detection/blob/main/Distilling-Object-Detectors/figures/table1.png?raw=true">    

<img src = "https://github.com/Sangh0/Object-Detection/blob/main/Distilling-Object-Detectors/figures/table2.png?raw=true">    

<img src = "https://github.com/Sangh0/Object-Detection/blob/main/Distilling-Object-Detectors/figures/table3.png?raw=true">  

<img src = "https://github.com/Sangh0/Object-Detection/blob/main/Distilling-Object-Detectors/figures/table4.png?raw=true">  

<img src = "https://github.com/Sangh0/Object-Detection/blob/main/Distilling-Object-Detectors/figures/table5.png?raw=true">    

### Analysis  
#### Visualization of imitation mask  
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/Distilling-Object-Detectors/figures/figure3.png?raw=true">  

- figure 3은 imitation mask를 시각화해 나타낸 이미지다  
- 여기에서 b, c, d는 각각 $\psi=0.2$, $\psi=0.5$, $\psi=0.8$을 사용해 생성한 mask이고 e와 f는 각각 constant threshold $F=0.5$, $F=0.8$을 사용해 생성한 mask이다  

#### Qualitative performance gain from imitation  
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/Distilling-Object-Detectors/figures/figure4.png?raw=true">  

- figure 4에서 1행의 이미지는 ground truth supervision만을 사용해 훈련한 student network의 output이고 2행의 이미지는 imitated student model의 output이다  
- 1행의 output은 하나의 object에 대해 여러 개의 box를 예측하고 존재하지 않는 object를 예측하는 등 좋지 않은 성능을 보여준 것에 비해 2행의 output은 비교적 정교한 예측 결과를 보여주고 있다  
- 여기에서 흥미로운 점은 2행의 첫 번째 이미지의 dog에 대한 confidence score는 0.32이다  
- 이때 teacher의 score를 살펴보면 0.38로 teacher의 knowledge를 student에 재대로 transfer되었다는 것을 알 수 있다  
- 이외에도 localization, 중복 예측 등을 고려하면 imitation을 사용한 것이 더 성능이 우수한 것을 알 수 있다  

<img src = "https://github.com/Sangh0/Object-Detection/blob/main/Distilling-Object-Detectors/figures/figure5.png?raw=true">  

- figure 5에서 왼쪽은 raw student model, 오른쪽은 imitated student model이다  
- correct score, localization score, background error percent 등을 살펴보면 imitated student network가 더 좋은 결과를 보여주고 있다  

#### Varying $\psi$ for generating mask  
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/Distilling-Object-Detectors/figures/figure6.png?raw=true">  

- $\psi=0$일 때, 즉 full feature imitation을 사용할 때 결과를 보면 mAP가 굉장히 낮음을 알 수 있으며 가장 적절한 thresholding factor는 $\psi=0.5$이다  

#### Per-channel variance of high level response  
- full feature imitation이 왜 성능을 악화시키는지 이해하기 위해 trained teacher model의 channel 당 variance를 살펴본다  
- figure 6의 b, c를 살펴보면 outside mask의 variance가 within mask의 variance가 더 큰 것을 알 수 있으며 이는 outside의 feature가 background의 noisy가 포함되어 있는 것으로 해석할 수 있다  
- 또한 우리는 full feature imitation의 loss value가 thresholding factor를 사용한 imitation feature보다 10배가 더 많은 것으로 나타났다  

## Conclusion  
- 우리는 fine-grained feature imitation method를 개발했으며 이는 object detection task는 local feature를 잡아내는 것이 더 중요하단 특성을 이용했다  
- 이를 통해 classification task에서만 사용되던 knowledge distillation method를 object detection에도 적용했으며 only supervision method보다 좋은 성능을 보여준다  
- prunning, quantization에서 유용하게 쓰일 수 있는 method가 되길 바란다